In [1]:
import requests
import json
from pprint import pprint
from config import zillow_key
import pandas as pd


In [2]:
#The url is from Nasdaq data sample
#In the first section

base_url = f'https://data.nasdaq.com/api/v3/datatables/ZILLOW/REGIONS?api_key={zillow_key}'
response = requests.get(base_url)
data = response.json()
filter_string = "Phoenix-Mesa-Chandler, AZ;Phoenix;Maricopa County"
filtered_data = [
    entry for entry in data['datatable']['data']
    if filter_string in entry[2]
]
pprint(filtered_data)


[['94915', 'zip', '85339;AZ;Phoenix-Mesa-Chandler, AZ;Phoenix;Maricopa County'],
 ['94892', 'zip', '85310;AZ;Phoenix-Mesa-Chandler, AZ;Phoenix;Maricopa County'],
 ['94846', 'zip', '85254;AZ;Phoenix-Mesa-Chandler, AZ;Phoenix;Maricopa County'],
 ['94791', 'zip', '85085;AZ;Phoenix-Mesa-Chandler, AZ;Phoenix;Maricopa County'],
 ['94767', 'zip', '85054;AZ;Phoenix-Mesa-Chandler, AZ;Phoenix;Maricopa County'],
 ['94766', 'zip', '85053;AZ;Phoenix-Mesa-Chandler, AZ;Phoenix;Maricopa County'],
 ['94765', 'zip', '85051;AZ;Phoenix-Mesa-Chandler, AZ;Phoenix;Maricopa County'],
 ['94764', 'zip', '85050;AZ;Phoenix-Mesa-Chandler, AZ;Phoenix;Maricopa County'],
 ['94763', 'zip', '85048;AZ;Phoenix-Mesa-Chandler, AZ;Phoenix;Maricopa County'],
 ['94761', 'zip', '85045;AZ;Phoenix-Mesa-Chandler, AZ;Phoenix;Maricopa County'],
 ['94760', 'zip', '85044;AZ;Phoenix-Mesa-Chandler, AZ;Phoenix;Maricopa County'],
 ['94759', 'zip', '85043;AZ;Phoenix-Mesa-Chandler, AZ;Phoenix;Maricopa County'],
 ['94758', 'zip', '85042;AZ;

In [3]:
regions_columns = ['region_id', 'region_type', 'region']
region_zillow_df = pd.DataFrame(filtered_data, columns=regions_columns)
region_zillow_df.to_csv('zillow_region.csv', index=False)


In [4]:
extracted_ids = [entry[0] for entry in filtered_data]
zip_ids = [[entry[0], entry[2]] for entry in filtered_data]
pprint(zip_ids)


[['94915', '85339;AZ;Phoenix-Mesa-Chandler, AZ;Phoenix;Maricopa County'],
 ['94892', '85310;AZ;Phoenix-Mesa-Chandler, AZ;Phoenix;Maricopa County'],
 ['94846', '85254;AZ;Phoenix-Mesa-Chandler, AZ;Phoenix;Maricopa County'],
 ['94791', '85085;AZ;Phoenix-Mesa-Chandler, AZ;Phoenix;Maricopa County'],
 ['94767', '85054;AZ;Phoenix-Mesa-Chandler, AZ;Phoenix;Maricopa County'],
 ['94766', '85053;AZ;Phoenix-Mesa-Chandler, AZ;Phoenix;Maricopa County'],
 ['94765', '85051;AZ;Phoenix-Mesa-Chandler, AZ;Phoenix;Maricopa County'],
 ['94764', '85050;AZ;Phoenix-Mesa-Chandler, AZ;Phoenix;Maricopa County'],
 ['94763', '85048;AZ;Phoenix-Mesa-Chandler, AZ;Phoenix;Maricopa County'],
 ['94761', '85045;AZ;Phoenix-Mesa-Chandler, AZ;Phoenix;Maricopa County'],
 ['94760', '85044;AZ;Phoenix-Mesa-Chandler, AZ;Phoenix;Maricopa County'],
 ['94759', '85043;AZ;Phoenix-Mesa-Chandler, AZ;Phoenix;Maricopa County'],
 ['94758', '85042;AZ;Phoenix-Mesa-Chandler, AZ;Phoenix;Maricopa County'],
 ['94757', '85041;AZ;Phoenix-Mesa-Chan

In [5]:
location_map = {entry[0]: entry[1] for entry in zip_ids}
indicators = ["ZSFH"]
region_ids = extracted_ids
second_url = "https://data.nasdaq.com/api/v3/datatables/ZILLOW/DATA"
all_data = []
for region_id in region_ids:
    for indicator in indicators:
        url = f"{second_url}?indicator_id={indicator}&region_id={region_id}&api_key={zillow_key}"
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            for item in data['datatable']['data']:
                full_location = location_map.get(region_id, "Location not found")
                zip_code = full_location.split(';')[0]
                item.append(zip_code)
                all_data.append(item)
        else:
            print(f"Failed to retrieve data for region_id {region_id}, indicator {indicator}")
pprint(all_data)


[['ZSFH', '94915', '2024-05-31', 447526.752889363, '85339'],
 ['ZSFH', '94915', '2024-04-30', 446274.306656896, '85339'],
 ['ZSFH', '94915', '2024-03-31', 444504.151743763, '85339'],
 ['ZSFH', '94915', '2024-02-29', 442772.869101603, '85339'],
 ['ZSFH', '94915', '2024-01-31', 441626.76275603, '85339'],
 ['ZSFH', '94915', '2023-12-31', 440806.589103963, '85339'],
 ['ZSFH', '94915', '2023-11-30', 439703.665869907, '85339'],
 ['ZSFH', '94915', '2023-10-31', 438308.261894117, '85339'],
 ['ZSFH', '94915', '2023-09-30', 436786.161501141, '85339'],
 ['ZSFH', '94915', '2023-08-31', 434962.569495565, '85339'],
 ['ZSFH', '94915', '2023-07-31', 432427.693546087, '85339'],
 ['ZSFH', '94915', '2023-06-30', 429708.231521726, '85339'],
 ['ZSFH', '94915', '2023-05-31', 427755.842342049, '85339'],
 ['ZSFH', '94915', '2023-04-30', 427129.219419957, '85339'],
 ['ZSFH', '94915', '2023-03-31', 428648.915834882, '85339'],
 ['ZSFH', '94915', '2023-02-28', 432654.671140304, '85339'],
 ['ZSFH', '94915', '2023-

In [7]:
columns = ['indicator_id', 'region_id', 'date', 'value', 'zip']
zillow_df = pd.DataFrame(all_data, columns=columns)
zillow_df['date'] = pd.to_datetime(zillow_df['date'])
zillow_df['value'] = zillow_df['value'].astype(float).round(2)
zillow_df.dtypes

indicator_id            object
region_id               object
date            datetime64[ns]
value                  float64
zip                     object
dtype: object

In [8]:
dates_of_interest = pd.to_datetime(['2006-12-31', '2009-12-31', '2019-12-31', '2023-12-31'])
date_df = zillow_df[zillow_df['date'].isin(dates_of_interest)]
date_df['value'] = date_df['value'].astype(float).round(2)
print(date_df)

      indicator_id region_id       date      value    zip
5             ZSFH     94915 2023-12-31  440806.59  85339
53            ZSFH     94915 2019-12-31  272393.34  85339
173           ZSFH     94915 2009-12-31  139760.01  85339
209           ZSFH     94915 2006-12-31  318938.46  85339
346           ZSFH     94892 2023-12-31  547504.34  85310
...            ...       ...        ...        ...    ...
13806         ZSFH     94720 2006-12-31  219898.28  85004
13943         ZSFH     94719 2023-12-31  678744.35  85003
13991         ZSFH     94719 2019-12-31  433846.82  85003
14111         ZSFH     94719 2009-12-31  234651.78  85003
14147         ZSFH     94719 2006-12-31  378267.58  85003

[172 rows x 5 columns]


C:\Users\AndBe\AppData\Local\Temp\ipykernel_13328\1329327120.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  date_df['value'] = date_df['value'].astype(float).round(2)


In [26]:
filtered_df = zillow_df[zillow_df['date'].isin(dates_of_interest)]
dates_df = filtered_df[filtered_df['date'].isin(pd.to_datetime(dates_of_interest))]
pivot_df = dates_df.pivot_table(index='zip', columns='date', values='value', aggfunc='mean')
pivot_df.columns = [date.year for date in pivot_df.columns]

if {2006, 2009, 2019, 2023}.issubset(pivot_df.columns):
    pivot_df['RecPr'] = pivot_df[2009] - pivot_df[2006]
    pivot_df['RevPr'] = pivot_df[2019] - pivot_df[2009]
    pivot_df['CovPr'] = pivot_df[2023] - pivot_df[2019]
    pivot_df['TotPr'] = pivot_df[2023] - pivot_df[2006]

    pivot_df['RecPc'] = ((pivot_df[2009] - pivot_df[2006]) / pivot_df[2006] * 100).round(2)
    pivot_df['RevPc'] = ((pivot_df[2019] - pivot_df[2009]) / pivot_df[2009] * 100).round(2)
    pivot_df['CovPc'] = ((pivot_df[2023] - pivot_df[2019]) / pivot_df[2019] * 100).round(2)
    pivot_df['TotPc'] = ((pivot_df[2023] - pivot_df[2006]) / pivot_df[2006] * 100).round(2)
pivot_df.reset_index(inplace=True)
pivot_df.to_csv('Zillow_final.csv', index=False)
pivot_df

,zip,2006,2009,2019,2023,RecPr,RevPr,CovPr,TotPr,RecPc,RevPc,CovPc,TotPc
0,85003,378267.58,234651.78,433846.82,678744.35,-143615.80,199195.04,244897.53,300476.77,-37.97,84.89,56.45,79.43
1,85004,219898.28,132668.62,304682.20,472755.48,-87229.66,172013.58,168073.28,252857.20,-39.67,129.66,55.16,114.99
2,85006,177278.87,90798.43,280123.98,442107.70,-86480.44,189325.55,161983.72,264828.83,-48.78,208.51,57.83,149.39
3,85007,219012.35,110243.62,273038.93,432713.79,-108768.73,162795.31,159674.86,213701.44,-49.66,147.67,58.48,97.58
4,85008,159335.33,90275.31,259682.50,402763.21,-69060.02,169407.19,143080.71,243427.88,-43.34,187.66,55.10,152.78
5,85009,138699.34,53452.32,180267.14,299231.53,-85247.02,126814.82,118964.39,160532.19,-61.46,237.25,65.99,115.74
6,85012,495535.27,332147.70,562607.90,877608.90,-163387.57,230460.20,315001.00,382073.63,-32.97,69.38,55.99,77.10
7,85013,274790.10,166136.00,361832.55,552613.19,-108654.10,195696.55,190780.64,277823.09,-39.54,117.79,52.73,101.10
8,85014,256275.75,151797.16,343960.49,523457.29,-104478.59,192163.33,179496.80,267181.54,-40.77,126.59,52.19,104.26
9,85015,184296.49,88807.17,242957.72,388039.82,-95489.32,154150.55,145082.10,203743.33,-51.81,173.58,59.71,110.55


In [ ]:
third_url = f'https://data.nasdaq.com/api/v3/datatables/ZILLOW/INDICATORS?api_key={zillow_key}'
response_indicators = requests.get(third_url)
data_indicators = response_indicators.json()
desired_indicators = ["ZSFH"]

filtered_indicators = [
    entry for entry in data_indicators['datatable']['data']
    if entry[0] in desired_indicators
]

pprint(filtered_indicators)


In [ ]:
indicators_columns = ['indicators_id', 'indicators', 'category']
indicators_zillow_df = pd.DataFrame(filtered_indicators, columns=indicators_columns)
indicators_zillow_df.to_csv('zillow_indicators.csv', index=False)